# Predicting Movie Attendance

This Jupyter notebook tests different classifiers/regressors from a sold movie tickets dataset. The goal is to forecast at daily level with at least one week in advance. These results will help the Operations Department with day-to-day inventory management that could benefit their selling concessions activities.

## Abstract
This work presents the methodology to predict movie attendance based on ticket sales and types of movies. Various techniques were used and the random forest approach performed better when comparing R^2 results. Movie tickets sales are from Cinepolis, and the dataset was obtained from 2016 to 2017. We used the following predictor variables: movie studios, type, exposure; month, day of the week, type of day, weekday / weekend and budget to construct the model.
We selected random forest approach after studing the dataset because we did not have to deselect variables and it is a technique the does not overfit the data. The random forest prediction was compared against another regressors. Also, a comparative analysis against an ARIMA model tells us that forecasting with this model reflects the impact of the variables on the movie attendance.

## 1) Introduction
Movie forecast attendance is needed to plan staff and estimate consessions. Numerous predictor variables come to mind from both academic literature and empirical knowledge. Because there might be a large set of predictor variables, it tends to overfitting. One way to avoid it, is to use a prediction technique that minimizes it. Random forest satisfies this requirement. The model is employed in this analysis purposely to avoid overfitting the data.

### 1.1) Data
The data sets used are from Cinepolis. Three sets of data were used: ticket sales per day, movie revenue and dates in theaters combined with Mexico holidays. The ticket sales per day was already standarized so a binning strategy was selected.

### 1.2) Data preparation
The data was combined to take into account that each day has a week or strong attraction to theaters depending on the predictor variables. 

#### Library loading

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sns.set_style('whitegrid')
sns.set_context('poster')
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn

#### Read the dataset
The file sales_16_17 contains two year ticket sales data. The file sales_18 was constructed for prediction. Is one week after the recieved data.

In [4]:
sales = pd.read_csv('./data/sales_16_17.csv')
sales_pred = pd.read_csv('./data/sales_18.csv')
sales = sales.append(sales_pred, ignore_index=True)

#### Date transformation
Since the dataset has a date column, it can be passed in date format to work the model with it.

In [5]:
try:
    sales['Day'] = pd.to_datetime(sales['Day'], format='%d/%m/%Y')
except Exception as e:
    print(e)

The dataset can be desagregated considering month and day name. Also, the distinction between weekday and weekend was considered.

In [6]:
sales['month'] = sales['Day'].dt.month_name()
sales['day_name'] = sales['Day'].dt.day_name()
sales['day_type'] = np.where(sales['Day'].dt.dayofweek < 5, 'weekday', 'weekend')

The data set was re-indexed by cronological order.

In [7]:
sales = sales.sort_values(by=['Day'])
sales.head()

,Day,BOLETOS,month,day_name,day_type
407,2016-01-01,2.082259,January,Friday,weekday
359,2016-01-02,2.051443,January,Saturday,weekend
387,2016-01-03,2.097130,January,Sunday,weekend
413,2016-01-04,0.678215,January,Monday,weekday
419,2016-01-05,0.736246,January,Tuesday,weekday


Now the day type according to the Mexican calendar was loaded to work with it.

In [8]:
holidays = pd.read_csv('./data/holidays.csv')
holidays.head()

,Fecha,Día,Nombre,Tipo
0,01/01/2016,Viernes,Año Nuevo,Descanso obligatorio
1,01/02/2016,Sábado,Vacaciones de invierno,Vacaciones escolares
2,01/03/2016,Domingo,Vacaciones de invierno,Vacaciones escolares
3,01/04/2016,Lunes,Vacaciones de invierno,Vacaciones escolares
4,01/05/2016,Martes,Vacaciones de invierno,Vacaciones escolares


Also the dataset contain a day column that can be used in date format.

In [9]:
try:
    holidays['Fecha'] = pd.to_datetime(holidays['Fecha'], format='%m/%d/%Y')
    #df.open_date.apply(lambda d: datetime.strptime(d, "%Y-%m-%d"))
except Exception as e:
    print(e)
holidays.head(7)

,Fecha,Día,Nombre,Tipo
0,2016-01-01,Viernes,Año Nuevo,Descanso obligatorio
1,2016-01-02,Sábado,Vacaciones de invierno,Vacaciones escolares
2,2016-01-03,Domingo,Vacaciones de invierno,Vacaciones escolares
3,2016-01-04,Lunes,Vacaciones de invierno,Vacaciones escolares
4,2016-01-05,Martes,Vacaciones de invierno,Vacaciones escolares
5,2016-01-06,Miércoles,Vacaciones de invierno,Vacaciones escolares
6,2016-02-01,Lunes,Día de la Constitución Méxicana,Descanso obligatorio


In [10]:
holidays.rename(columns={'Fecha': 'Day'}, inplace=True)
holidays.head(7)

,Day,Día,Nombre,Tipo
0,2016-01-01,Viernes,Año Nuevo,Descanso obligatorio
1,2016-01-02,Sábado,Vacaciones de invierno,Vacaciones escolares
2,2016-01-03,Domingo,Vacaciones de invierno,Vacaciones escolares
3,2016-01-04,Lunes,Vacaciones de invierno,Vacaciones escolares
4,2016-01-05,Martes,Vacaciones de invierno,Vacaciones escolares
5,2016-01-06,Miércoles,Vacaciones de invierno,Vacaciones escolares
6,2016-02-01,Lunes,Día de la Constitución Méxicana,Descanso obligatorio


Having made the day names transformation. The data set are combined to categorize each day according to the holiday or type of day according to the Mexican Calendar.

In [11]:
result = pd.merge(sales, holidays, how='outer', on='Day')
result.head(7)

,Day,BOLETOS,month,day_name,day_type,Día,Nombre,Tipo
0,2016-01-01,2.082259,January,Friday,weekday,Viernes,Año Nuevo,Descanso obligatorio
1,2016-01-02,2.051443,January,Saturday,weekend,Sábado,Vacaciones de invierno,Vacaciones escolares
2,2016-01-03,2.097130,January,Sunday,weekend,Domingo,Vacaciones de invierno,Vacaciones escolares
3,2016-01-04,0.678215,January,Monday,weekday,Lunes,Vacaciones de invierno,Vacaciones escolares
4,2016-01-05,0.736246,January,Tuesday,weekday,Martes,Vacaciones de invierno,Vacaciones escolares
5,2016-01-06,0.820632,January,Wednesday,weekday,Miércoles,Vacaciones de invierno,Vacaciones escolares
6,2016-01-07,0.373011,January,Thursday,weekday,NaN,NaN,NaN


In [12]:
print(result['Tipo'].unique())

['Descanso obligatorio' 'Vacaciones escolares' nan 'No obligatorio']


The type of day was changed into a machine readable type. This helps to categorize, bin and make dummy variables.

In [13]:
day_dic = {'Descanso obligatorio': 'RDAY', 'Vacaciones escolares': 'HDAY', 'No obligatorio': 'LDAY', np.nan: 'NDAY'}
result = result.replace({'Tipo': day_dic})
result = result.rename(columns={'Tipo': 'day_tipo'})

In [14]:
result = result.drop(columns=['Día', 'Nombre'])

The month, day, weekday/weekend and day type according to the Mexican calendar were used to make dummy variables.

In [15]:
df_mon_name = pd.get_dummies(result['month'])
df_day_name = pd.get_dummies(result['day_name'])
df_day_type = pd.get_dummies(result['day_type'])
df_day_tipo = pd.get_dummies(result['day_tipo'])

In [16]:
result = pd.concat([result, df_mon_name, df_day_name, df_day_type, df_day_tipo], axis=1)


In [17]:
#result = result.drop(columns=['day_name', 'day_type', 'day_tipo'])
result.head(15)

,Day,BOLETOS,month,day_name,day_type,day_tipo,April,August,December,February,...,Sunday,Thursday,Tuesday,Wednesday,weekday,weekend,HDAY,LDAY,NDAY,RDAY
0,2016-01-01,2.082259,January,Friday,weekday,RDAY,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
1,2016-01-02,2.051443,January,Saturday,weekend,HDAY,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
2,2016-01-03,2.097130,January,Sunday,weekend,HDAY,0,0,0,0,...,1,0,0,0,0,1,1,0,0,0
3,2016-01-04,0.678215,January,Monday,weekday,HDAY,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
4,2016-01-05,0.736246,January,Tuesday,weekday,HDAY,0,0,0,0,...,0,0,1,0,1,0,1,0,0,0
5,2016-01-06,0.820632,January,Wednesday,weekday,HDAY,0,0,0,0,...,0,0,0,1,1,0,1,0,0,0
6,2016-01-07,0.373011,January,Thursday,weekday,NDAY,0,0,0,0,...,0,1,0,0,1,0,0,0,1,0
7,2016-01-08,1.003438,January,Friday,weekday,NDAY,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
8,2016-01-09,1.650276,January,Saturday,weekend,NDAY,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
9,2016-01-10,2.011941,January,Sunday,weekend,NDAY,0,0,0,0,...,1,0,0,0,0,1,0,0,1,0


The results are exported to a CSV file.

In [18]:
result.to_csv('./data/sales_by_day_bis.csv', index=False)

### 2) Movie Data Feature Engineering
The dataset containing the movies featuring in theaters around those dates were loaded and transformed in order to be included into the ticket sales dataframe.

#### Read the datafiles

In [19]:
df = pd.read_csv('./data/movie_data(copy).csv')
df.head(15)

,studio,film,total_gross,domestic_gross,international_gross,runtime_min,budget,rating,num_theaters,days_in_theater,open_date
0,Buena Vista,Guardians of the Galaxy,773328629,333176600,440152029,141,170000000,PG-13,4088,175,2014-08-01
1,Buena Vista,"Alexander and the Terrible, Horrible, No Good,...",101379287,66954149,34425138,81,28000000,PG,3117,140,2014-10-10
2,Buena Vista,Big Hero 6,657818612,222527828,435290784,108,165000000,PG,3773,203,2014-11-07
3,WB,Horrible Bosses 2,107670357,54445357,53225000,108,42000000,R,3400,86,2014-11-26
4,WB,The Hobbit: Battle of the Five Armies,956019788,255119788,700900000,144,250000000,PG-13,3875,107,2014-12-17
5,WB,American Sniper,547426372,350126372,197300000,152,58800000,R,3885,183,2014-12-25
6,Buena Vista,Into the Woods,213116401,128002372,85114029,144,50000000,PG,2833,113,2014-12-25
7,20th Century Fox,Taken 3,326479141,89256424,237222717,109,48000000,PG-13,3594,126,2015-01-09
8,Sony,Still Alice,43884652,18754371,25130281,101,5000000,PG-13,1318,0,2015-01-16
9,Universal,The Boy Next Door,52425855,35423380,17002475,91,4000000,R,2615,49,2015-01-23


Read the data file and make the closing date as the sum of opening date plus the days in theaters.

In [20]:
try:
    df['open_date'] = pd.to_datetime(df['open_date'], format='%Y-%m-%d')
    #df.open_date.apply(lambda d: datetime.strptime(d, "%Y-%m-%d"))
except Exception as e:
    print(e)

In [21]:
temp = df['days_in_theater'].apply(np.ceil).apply(lambda x: pd.Timedelta(x, unit='D'))
df['close_date'] = pd.to_datetime(df['open_date']) + temp

Consideration:
Absent values in budget or international gross are set to zero.
The variable "mov_budget" is the divition between the international gross (since the analysis is for Mexico) and budget.

In [22]:
mask = (df['international_gross'] == 0)| (df['budget'] == 0)
df['mov_budget'] = df['international_gross'].div(df['budget'], fill_value=0).where(~mask,0)

### 2.1) Binning
Since the ticket sales are Standarized, that gives us the opportunity to include some movie categories according to budget, exposure (number of theaters where the movies were played), extent (number of days when movies were played) and the film studio.
The folowing categories were used for mov_budget:
N = None with values between -1 and 1.
L = Low with values between 1 and 2
M = Medium with values between 2 and 5.
H = High with values above 5 (with a limit of 250)
The number of theaters is binned using the following scale into the following variable "mov_exp" (movie exposure):
L = Low with values between -1 and 1500.
M = Medium with values between 1500 and 3500.
H = High with values between 3500 and 5000.
The number of days is binned using the following scale into the following variable "mov_ext" (movie extent):
L = Low with values between -1 and 100.
M = Medium with values between 100 and 200.
H = High with values between 200 and 300.
And the Movie Studios are abreviated.

In [23]:
bins_0 = [-1, 1, 2, 5, 250]
labels_0 = ['N','L','M','H']
df['mov_budget'] = pd.cut(df['mov_budget'], bins=bins_0, labels=labels_0)
df['mov_budget'] = df['mov_budget'].astype(str)

In [24]:
bins_1 = [-1, 1500, 3500, 5000]
labels_1 = ['L','M','H']
df['mov_exp'] = pd.cut(df['num_theaters'], bins=bins_1, labels=labels_1)
df['mov_exp'] = df['mov_exp'].astype(str)

In [25]:
bins_2 = [-1, 100, 200, 300]
labels_2 = ['L','M','H']
df['mov_ext'] = pd.cut(df['days_in_theater'], bins=bins_2, labels=labels_2)
df['mov_ext'] = df['mov_ext'].astype(str)

In [26]:
dict_studio = {'Buena Vista':'BV', '20th Century Fox': 'FX', 'Sony': 'SO', 'Universal':'UN', 'Paramount':'PT'}
df = df.replace({'studio': dict_studio})
df['studio'] = df['studio'].astype(str)
df.head()

,studio,film,total_gross,domestic_gross,international_gross,runtime_min,budget,rating,num_theaters,days_in_theater,open_date,close_date,mov_budget,mov_exp,mov_ext
0,BV,Guardians of the Galaxy,773328629,333176600,440152029,141,170000000,PG-13,4088,175,2014-08-01,2015-01-23,M,H,M
1,BV,"Alexander and the Terrible, Horrible, No Good,...",101379287,66954149,34425138,81,28000000,PG,3117,140,2014-10-10,2015-02-27,L,M,M
2,BV,Big Hero 6,657818612,222527828,435290784,108,165000000,PG,3773,203,2014-11-07,2015-05-29,M,H,H
3,WB,Horrible Bosses 2,107670357,54445357,53225000,108,42000000,R,3400,86,2014-11-26,2015-02-20,L,M,L
4,WB,The Hobbit: Battle of the Five Armies,956019788,255119788,700900000,144,250000000,PG-13,3875,107,2014-12-17,2015-04-03,M,H,M


A basic check to se if there is a missing value.

In [27]:
print(df.isnull().sum())

studio                 0
film                   0
total_gross            0
domestic_gross         0
international_gross    0
runtime_min            0
budget                 0
rating                 0
num_theaters           0
days_in_theater        0
open_date              0
close_date             0
mov_budget             0
mov_exp                0
mov_ext                0
dtype: int64


The new colums where the united to create one category from multiple characteristics. This create a code for each type of movie.

In [28]:
df['mov_cat'] = df[['mov_budget', 'mov_exp', 'mov_ext']].apply(lambda x: ''.join(x), axis=1)

In [29]:
print(df.mov_cat.unique())
print(df.dtypes)

['MHM' 'LMM' 'MHH' 'LML' 'HLL' 'MML' 'NML' 'HHL' 'MMM' 'NMM' 'LHM' 'HML'
 'MHL' 'NHM' 'HHM' 'NHL' 'LHL' 'NLL' 'HMM' 'NLM' 'HMH']
studio                         object
film                           object
total_gross                     int64
domestic_gross                  int64
international_gross             int64
runtime_min                     int64
budget                          int64
rating                         object
num_theaters                    int64
days_in_theater                 int64
open_date              datetime64[ns]
close_date             datetime64[ns]
mov_budget                     object
mov_exp                        object
mov_ext                        object
mov_cat                        object
dtype: object


### 2.2) Combining sets
Combining the categories is used to reflect that each day had a particular movie type, from a specific studio played on a specific date. Therefore for each day the previous categories were joined to the sales dataset.

In [30]:
sales = pd.read_csv('./data/sales_by_day_bis.csv')
sales['Day'] = pd.to_datetime(sales['Day'])

In [31]:
studios = ['BV', 'WB', 'FX', 'SO', 'UN', 'PT']
for studio in studios:
    sales[studio] = 0
    for ind in sales.index:
        sales[studio][ind] = (((sales['Day'][ind] >= df['open_date']) & (sales['Day'][ind] <= df['close_date'])) & (df['studio'] == studio)).astype(int).sum()

In [32]:
movcategories = ['MHM', 'LMM', 'MHH', 'LML', 'HLL', 'MML', 'NML', 'HHL', 'MMM', \
          'NMM', 'LHM', 'HML', 'MHL', 'NHM', 'HHM', 'NHL', 'LHL', 'NLL', \
          'HMM', 'NLM', 'HMH']
for movcategory in movcategories:
    sales[movcategory] = 0
    for ind in sales.index:
        sales[movcategory][ind] = (((sales['Day'][ind] >= df['open_date']) & (sales['Day'][ind] <= df['close_date'])) & (df['mov_cat'] == movcategory)).astype(int).sum()

In [33]:
sales.shape


(738, 58)

There train dataset (from Cinepolis) and the test dataset (will be used for prediction) are separated since the recieved the same treatment for model construction purposes.

In [34]:
sales_train = sales.iloc[:731]
sales_test = sales.iloc[-7:]

In [35]:
sales_train.to_csv('./data/sales_train.csv', index=False)
sales_test.to_csv('./data/sales_test.csv', index=False)

Since this analysis is used to predict a week in advance, the test CSV is the first week of 2018.